# Thermal Oxidation Simulation: Deal-Grove & Massoud Models

This notebook demonstrates thermal oxidation simulation using the Deal-Grove model with Massoud thin-oxide corrections.

## Contents
1. Deal-Grove Model Basics
2. Temperature Dependence
3. Dry vs Wet Oxidation
4. Massoud Thin-Oxide Correction
5. Inverse Problem: Time to Target Thickness
6. API Usage Examples

In [ ]:
import sys
sys.path.insert(0, '..')

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from core import deal_grove, massoud

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 11

print("Modules loaded successfully!")

## 1. Deal-Grove Model Basics

The Deal-Grove model describes oxide growth with:

$$x_{ox}^2 + A \cdot x_{ox} = B \cdot (t + \tau)$$

Where:
- $x_{ox}$: oxide thickness
- $A$: linear rate constant
- $B$: parabolic rate constant
- $t$: oxidation time
- $\tau$: time shift for initial oxide

In [ ]:
# Example: Dry oxidation at 1000°C
T = 1000  # Celsius
ambient = 'dry'

# Get rate constants
B, B_over_A = deal_grove.get_rate_constants(T, ambient)
A = B / B_over_A

print(f"Deal-Grove Parameters at {T}°C ({ambient} oxidation):")
print(f"  B       = {B:.2e} μm²/hr")
print(f"  B/A     = {B_over_A:.2e} μm/hr")
print(f"  A       = {A:.4f} μm")
print(f"\nLinear regime (x << A):  growth rate ≈ B/A = {B_over_A:.4f} μm/hr")
print(f"Parabolic regime (x >> A): growth rate ≈ B/(2x)")

In [ ]:
# Time evolution
times = np.linspace(0.1, 10, 100)  # hours
thickness = deal_grove.thickness_at_time(times, T, ambient)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Linear scale
ax1.plot(times, thickness * 1000, 'b-', linewidth=2)
ax1.set_xlabel('Time (hours)', fontsize=12)
ax1.set_ylabel('Oxide Thickness (nm)', fontsize=12)
ax1.set_title(f'Dry Oxidation at {T}°C - Linear Scale', fontsize=13, fontweight='bold')
ax1.grid(True, alpha=0.3)

# Log-log scale
ax2.loglog(times, thickness * 1000, 'b-', linewidth=2)
ax2.set_xlabel('Time (hours)', fontsize=12)
ax2.set_ylabel('Oxide Thickness (nm)', fontsize=12)
ax2.set_title(f'Dry Oxidation at {T}°C - Log-Log Scale', fontsize=13, fontweight='bold')
ax2.grid(True, alpha=0.3, which='both')

# Add regime markers
ax2.axhline(y=A*1000, color='r', linestyle='--', alpha=0.5, label=f'A = {A*1000:.1f} nm')
ax2.legend()

plt.tight_layout()
plt.show()

print("\nNote: Linear regime dominates for x << A, parabolic for x >> A")

## 2. Temperature Dependence

Rate constants follow Arrhenius behavior:

$$k = k_0 \exp\left(-\frac{E_a}{k_B T}\right)$$

In [ ]:
# Compare different temperatures
temperatures = [900, 1000, 1100, 1200]
times = np.linspace(0.1, 5, 100)

fig, ax = plt.subplots(figsize=(12, 7))

for T in temperatures:
    thickness = deal_grove.thickness_at_time(times, T, 'dry')
    ax.plot(times, thickness * 1000, linewidth=2, label=f'{T}°C')

ax.set_xlabel('Time (hours)', fontsize=12)
ax.set_ylabel('Oxide Thickness (nm)', fontsize=12)
ax.set_title('Temperature Dependence of Dry Oxidation', fontsize=13, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Show rate constants vs temperature
print("\nRate Constants vs Temperature (Dry Oxidation):")
print(f"{'T (°C)':<10} {'B (μm²/hr)':<15} {'B/A (μm/hr)':<15} {'A (nm)':<10}")
print("-" * 55)
for T in temperatures:
    B, B_over_A = deal_grove.get_rate_constants(T, 'dry')
    A = B / B_over_A
    print(f"{T:<10} {B:<15.2e} {B_over_A:<15.2e} {A*1000:<10.2f}")

## 3. Dry vs Wet Oxidation

Wet oxidation (H₂O) is much faster than dry oxidation (O₂).

In [ ]:
# Compare dry and wet at same temperature
T = 1000
times = np.linspace(0.1, 5, 100)

thickness_dry = deal_grove.thickness_at_time(times, T, 'dry')
thickness_wet = deal_grove.thickness_at_time(times, T, 'wet')

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Linear scale
ax1.plot(times, thickness_dry * 1000, 'b-', linewidth=2, label='Dry O₂')
ax1.plot(times, thickness_wet * 1000, 'r-', linewidth=2, label='Wet H₂O')
ax1.set_xlabel('Time (hours)', fontsize=12)
ax1.set_ylabel('Oxide Thickness (nm)', fontsize=12)
ax1.set_title(f'Dry vs Wet Oxidation at {T}°C', fontsize=13, fontweight='bold')
ax1.legend(fontsize=11)
ax1.grid(True, alpha=0.3)

# Ratio
ratio = thickness_wet / thickness_dry
ax2.plot(times, ratio, 'g-', linewidth=2)
ax2.set_xlabel('Time (hours)', fontsize=12)
ax2.set_ylabel('Thickness Ratio (Wet/Dry)', fontsize=12)
ax2.set_title('Wet/Dry Thickness Ratio', fontsize=13, fontweight='bold')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Show rate constant comparison
B_dry, BA_dry = deal_grove.get_rate_constants(T, 'dry')
B_wet, BA_wet = deal_grove.get_rate_constants(T, 'wet')

print(f"\nRate Constant Comparison at {T}°C:")
print(f"  B_wet / B_dry   = {B_wet/B_dry:.1f}x")
print(f"  (B/A)_wet / (B/A)_dry = {BA_wet/BA_dry:.1f}x")
print(f"\nWet oxidation is ~{B_wet/B_dry:.0f}x faster!")

## 4. Massoud Thin-Oxide Correction

For thin oxides (<~70 nm), Deal-Grove underestimates growth rates. The Massoud correction adds:

$$x_{Massoud} = x_{DG} + C \exp\left(-\frac{x_{DG}}{L}\right)$$

In [ ]:
# Compare Deal-Grove and Massoud models
T = 900  # Lower temperature where thin-oxide effects are more pronounced
times = np.linspace(0.05, 3, 100)

# Calculate both models
thickness_dg = deal_grove.thickness_at_time(times, T, 'dry')
thickness_mass = massoud.thickness_with_correction(times, T, 'dry', apply_correction=True)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Thickness comparison
ax1.plot(times, thickness_dg * 1000, 'b--', linewidth=2, label='Deal-Grove')
ax1.plot(times, thickness_mass * 1000, 'r-', linewidth=2, label='Massoud')
ax1.set_xlabel('Time (hours)', fontsize=12)
ax1.set_ylabel('Oxide Thickness (nm)', fontsize=12)
ax1.set_title(f'Deal-Grove vs Massoud at {T}°C (Dry)', fontsize=13, fontweight='bold')
ax1.legend(fontsize=11)
ax1.grid(True, alpha=0.3)

# Correction magnitude
correction = (thickness_mass - thickness_dg) * 1000  # nm
ax2.plot(thickness_dg * 1000, correction, 'g-', linewidth=2)
ax2.set_xlabel('Deal-Grove Thickness (nm)', fontsize=12)
ax2.set_ylabel('Correction (nm)', fontsize=12)
ax2.set_title('Massoud Correction Magnitude', fontsize=13, fontweight='bold')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Show where correction is significant
C, L = massoud.get_correction_params(T, 'dry')
print(f"\nMassoud Parameters at {T}°C:")
print(f"  C = {C:.2f} nm (amplitude)")
print(f"  L = {L:.2f} nm (characteristic length)")
print(f"\nCorrection becomes negligible for x >> L ≈ {L:.1f} nm")

In [ ]:
# Show relative correction vs thickness
thicknesses_nm = np.logspace(0, 2.5, 100)  # 1 to ~300 nm
thicknesses_um = thicknesses_nm / 1000

corrections = [massoud.correction_magnitude(x, 900, 'dry') for x in thicknesses_um]
relative_corrections = [c / (x*1000) * 100 for c, x in zip(corrections, thicknesses_um)]

fig, ax = plt.subplots(figsize=(10, 6))
ax.semilogx(thicknesses_nm, relative_corrections, 'b-', linewidth=2)
ax.axhline(y=5, color='r', linestyle='--', alpha=0.5, label='5% threshold')
ax.set_xlabel('Oxide Thickness (nm)', fontsize=12)
ax.set_ylabel('Relative Correction (%)', fontsize=12)
ax.set_title('Significance of Massoud Correction', fontsize=13, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nCorrection is > 5% for thicknesses below ~50 nm")

## 5. Inverse Problem: Time to Target Thickness

Calculate the time required to grow oxide to a target thickness.

In [ ]:
# Calculate time to reach various target thicknesses
target_thicknesses_nm = [10, 20, 50, 100, 200, 500, 1000]  # nm
temperatures = [900, 1000, 1100]

print("Time Required to Reach Target Thickness\n")
print("="*80)

for ambient in ['dry', 'wet']:
    print(f"\n{ambient.upper()} OXIDATION")
    print("-"*80)
    print(f"{'Target (nm)':<12}", end='')
    for T in temperatures:
        print(f"{T}°C (hr)     ", end='')
    print()
    print("-"*80)
    
    for target_nm in target_thicknesses_nm:
        target_um = target_nm / 1000
        print(f"{target_nm:<12}", end='')
        for T in temperatures:
            t_mass = massoud.time_to_thickness_with_correction(
                target_um, T, ambient, apply_correction=True
            )
            if t_mass < 0.01:
                print(f"{t_mass*60:>8.2f} min   ", end='')
            else:
                print(f"{t_mass:>11.3f}   ", end='')
        print()

In [ ]:
# Visualize inverse problem: time contours
target_thicknesses = np.linspace(10, 500, 50)  # nm
temperatures = np.linspace(900, 1200, 50)  # C

T_grid, X_grid = np.meshgrid(temperatures, target_thicknesses)
time_grid_dry = np.zeros_like(T_grid)
time_grid_wet = np.zeros_like(T_grid)

for i in range(len(target_thicknesses)):
    for j in range(len(temperatures)):
        x_um = target_thicknesses[i] / 1000
        T = temperatures[j]
        time_grid_dry[i, j] = massoud.time_to_thickness_with_correction(
            x_um, T, 'dry', apply_correction=True
        )
        time_grid_wet[i, j] = massoud.time_to_thickness_with_correction(
            x_um, T, 'wet', apply_correction=True
        )

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Dry oxidation
cs1 = ax1.contourf(T_grid, X_grid, time_grid_dry, levels=20, cmap='viridis')
ax1.contour(T_grid, X_grid, time_grid_dry, levels=10, colors='white', alpha=0.3, linewidths=0.5)
cb1 = plt.colorbar(cs1, ax=ax1)
cb1.set_label('Time (hours)', fontsize=11)
ax1.set_xlabel('Temperature (°C)', fontsize=12)
ax1.set_ylabel('Target Thickness (nm)', fontsize=12)
ax1.set_title('Dry Oxidation: Time to Target', fontsize=13, fontweight='bold')

# Wet oxidation
cs2 = ax2.contourf(T_grid, X_grid, time_grid_wet, levels=20, cmap='viridis')
ax2.contour(T_grid, X_grid, time_grid_wet, levels=10, colors='white', alpha=0.3, linewidths=0.5)
cb2 = plt.colorbar(cs2, ax=ax2)
cb2.set_label('Time (hours)', fontsize=11)
ax2.set_xlabel('Temperature (°C)', fontsize=12)
ax2.set_ylabel('Target Thickness (nm)', fontsize=12)
ax2.set_title('Wet Oxidation: Time to Target', fontsize=13, fontweight='bold')

plt.tight_layout()
plt.show()

## 6. API Usage Examples

The simulation is also available via REST API.

In [ ]:
# Example API request (requires running server)
import json

# Sample request payload
api_request = {
    "temperature": 1000,
    "ambient": "dry",
    "time_points": [0.5, 1.0, 2.0, 4.0, 8.0],
    "pressure": 1.0,
    "initial_thickness": 0.0,
    "use_massoud": True,
    "target_thickness": 0.5  # μm
}

print("Example API Request:")
print(json.dumps(api_request, indent=2))

print("\n" + "="*60)
print("To use the API:")
print("1. Start server: python -m api.service")
print("2. POST to: http://localhost:8000/oxidation/simulate")
print("3. View docs: http://localhost:8000/docs")
print("="*60)

In [ ]:
# If API server is running, test it
try:
    import httpx
    
    # Try to connect to API
    with httpx.Client() as client:
        response = client.post(
            "http://localhost:8000/oxidation/simulate",
            json=api_request,
            timeout=10.0
        )
        
        if response.status_code == 200:
            result = response.json()
            print("API Response:")
            print(json.dumps(result, indent=2))
            
            # Plot results
            fig, ax = plt.subplots(figsize=(10, 6))
            ax.plot(result['time_points'], result['thickness_nm'], 'bo-', linewidth=2, markersize=8)
            ax.set_xlabel('Time (hours)', fontsize=12)
            ax.set_ylabel('Oxide Thickness (nm)', fontsize=12)
            ax.set_title('API Simulation Result', fontsize=13, fontweight='bold')
            ax.grid(True, alpha=0.3)
            plt.tight_layout()
            plt.show()
        else:
            print(f"API request failed with status {response.status_code}")
            
except Exception as e:
    print(f"API server not running or not accessible: {e}")
    print("Start the server with: python -m api.service")

## Summary

This notebook demonstrated:

1. **Deal-Grove Model**: Linear-parabolic oxidation kinetics
2. **Temperature Effects**: Arrhenius rate dependence
3. **Dry vs Wet**: ~10x faster growth with H₂O
4. **Massoud Correction**: Thin-oxide enhancement (<70 nm)
5. **Inverse Problem**: Time to reach target thickness
6. **API Integration**: REST API for simulation

### Key Takeaways

- Oxide growth transitions from linear (thin) to parabolic (thick) regimes
- Wet oxidation is ~10x faster than dry
- Massoud correction is important for oxides < 50-70 nm
- Temperature has exponential effect on growth rate
- Inverse solver enables process planning